In [1]:
# import packages
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from matplotlib.legend_handler import HandlerLine2D
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV


In [2]:
# load the data
house_data = pd.read_csv('Shortlisted Train - train.csv')
dt = pd.DataFrame(house_data)
#cut the data
#house_data.drop(house_data.iloc[:, 0:3], inplace=True, axis=1)
#display(house_data)
#house_data.drop(house_data.iloc[:, 1:35], inplace=True, axis=1)

df = dt[['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'LotShape', 'Neighborhood',
         'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 
         'Exterior1st', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtFinType1', 'BsmtFinSF1', 
         'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'GrLivArea', 'TotalFullBath', 
         'TotalHalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 
         'Fireplaces', 'GarageCars', 'PavedDrive', 'YrSold', 'SaleCondition', 'SalePrice']]
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,Neighborhood,BldgType,HouseStyle,OverallQual,OverallCond,...,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,GarageCars,PavedDrive,YrSold,SaleCondition,SalePrice
0,60,RL,65.0,8450,Reg,CollgCr,1Fam,2Story,7,5,...,3,1,Gd,8,0,2,Y,2008,Normal,208500
1,20,RL,80.0,9600,Reg,Veenker,1Fam,1Story,6,8,...,3,1,TA,6,1,2,Y,2007,Normal,181500
2,60,RL,68.0,11250,IR1,CollgCr,1Fam,2Story,7,5,...,3,1,Gd,6,1,2,Y,2008,Normal,223500
3,70,RL,60.0,9550,IR1,Crawfor,1Fam,2Story,7,5,...,3,1,Gd,7,1,3,Y,2006,Abnorml,140000
4,60,RL,84.0,14260,IR1,NoRidge,1Fam,2Story,8,5,...,4,1,Gd,9,1,3,Y,2008,Normal,250000


In [3]:
### Cleaning dataset for ML models -

#First: Replacing NAs:

#LotFrontage, fill in NAs with mean
df["LotFrontage"] = df["LotFrontage"].fillna(df["LotFrontage"].mean())

#BsmtFinType1, categorical basement feature, Na means there isn't a basement, so we can substitute with 'None'
df['BsmtFinType1'] = df['BsmtFinType1'].fillna('None')

C:\Users\deeks\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\deeks\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [4]:
#Converting categorical variables and misinterpreted features

#MSSubClass
df['MSSubClass'] = df['MSSubClass'].apply(str)

#Year
df['YrSold'] = df['YrSold'].astype(str)

#YearBuilt
df['YearBuilt'] = df['YearBuilt'].astype(str)

#YearRemodAdd
df['YearRemodAdd'] = df['YearRemodAdd'].astype(str)

C:\Users\deeks\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\deeks\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\deeks\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas

In [5]:
#Convert 'CentralAir' yes/no to 1/0
df['CentralAir'] = df['CentralAir'].replace("N", 0).replace("Y", 1)

#Labelencoder to scale as numeric (1 to n) for other categories 
from sklearn.preprocessing import LabelEncoder
cols = ('ExterCond','HeatingQC', 'KitchenQual', 'BsmtFinType1', 'BldgType',
        'LotShape', 'PavedDrive', 'YrSold', 'YearBuilt', 'YearRemodAdd', 'ExterQual')

# Process columns and apply LabelEncoder to transform 
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(df[c].values)) 
    df[c] = lbl.transform(list(df[c].values))

# Lotfrontage should be int, not float (data type retained because we filled in NAs previously) Convert:
df.LotFrontage = df.LotFrontage.astype(int)

#Check data types
df.dtypes

C:\Users\deeks\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\deeks\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\deeks\Anaconda3\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stab

MSSubClass       object
MSZoning         object
LotFrontage       int32
LotArea           int64
LotShape          int64
Neighborhood     object
BldgType          int64
HouseStyle       object
OverallQual       int64
OverallCond       int64
YearBuilt         int64
YearRemodAdd      int64
Exterior1st      object
ExterQual         int64
ExterCond         int64
Foundation       object
BsmtFinType1      int64
BsmtFinSF1        int64
TotalBsmtSF       int64
Heating          object
HeatingQC         int64
CentralAir        int64
GrLivArea         int64
TotalFullBath     int64
TotalHalfBath     int64
BedroomAbvGr      int64
KitchenAbvGr      int64
KitchenQual       int64
TotRmsAbvGrd      int64
Fireplaces        int64
GarageCars        int64
PavedDrive        int64
YrSold            int64
SaleCondition    object
SalePrice         int64
dtype: object

In [6]:
#First creating 2 new dataframes, d1 as all the predictors we are keeping, d2 as the target (price)
#  i.e.  dropped the bathroom columns for basement/above ground because we already have the total count

d1 = df[['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'LotShape', 'Neighborhood',
         'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 
        'ExterQual', 'Foundation', 'TotalBsmtSF',  'HeatingQC',  'GrLivArea', 'TotalFullBath', 
         'TotalHalfBath', 'KitchenQual', 'TotRmsAbvGrd', 
         'Fireplaces','YrSold', 'SaleCondition']]

d2 = df[['SalePrice']]

In [7]:
#SaleCondition - normal, bankrupt, family, etc. 
dummy_salecond = pd.get_dummies(d1["SaleCondition"],drop_first=True)

#MSZoning
dummy_zoning = pd.get_dummies(d1["MSZoning"],drop_first=True)

#Neighborhood
dummy_neighborhood = pd.get_dummies(d1["Neighborhood"],drop_first=True)

#HouseStyle
dummy_housestyle = pd.get_dummies(d1["HouseStyle"],drop_first=True)

#Exterior1st
#dummy_exterior = pd.get_dummies(d1["Exterior1st"],drop_first=True)

#Foundation
dummy_foundation = pd.get_dummies(d1["Foundation"],drop_first=True)

#Heating
#dummy_heating = pd.get_dummies(d1["Heating"],drop_first=True)

#MSSubClass
dummy_mssub = pd.get_dummies(d1["MSSubClass"],drop_first=True)


d1 = pd.concat([d1, dummy_salecond, dummy_zoning, dummy_neighborhood, dummy_housestyle, 
              # dummy_exterior, 
                dummy_foundation, 
                #dummy_heating, 
                dummy_mssub],axis=1)


In [8]:
# Remove original columns after one-hot completed (no longer needed)? But first column already dropped.
d1.drop(["HouseStyle",
         #"Heating",
         "SaleCondition", "MSZoning","Neighborhood", "HouseStyle", 
         #"Exterior1st", 
         "Foundation", "MSSubClass" ],axis=1,inplace=True)


# Dropping even more uncorrelated variables or avoid multicollinearity? Up to interpretation

In [9]:
d1.head()

,LotFrontage,LotArea,LotShape,BldgType,OverallQual,OverallCond,YearBuilt,YearRemodAdd,ExterQual,TotalBsmtSF,...,30,40,45,50,60,70,75,80,85,90
0,65,8450,3,0,7,5,104,53,2,856,...,0,0,0,0,1,0,0,0,0,0
1,80,9600,3,0,6,8,77,26,3,1262,...,0,0,0,0,0,0,0,0,0,0
2,68,11250,0,0,7,5,102,52,2,920,...,0,0,0,0,1,0,0,0,0,0
3,60,9550,0,0,7,5,19,20,3,756,...,0,0,0,0,0,1,0,0,0,0
4,84,14260,0,0,8,5,101,50,2,1145,...,0,0,0,0,1,0,0,0,0,0


In [10]:
d2.head()

,SalePrice
0,208500
1,181500
2,223500
3,140000
4,250000


In [11]:
from sklearn.model_selection import train_test_split  
train_x, test_x, train_y, test_y = train_test_split(d1, d2, test_size=0.20)

In [12]:
train = np.array(train_y)
train

array([[260000],
       [143000],
       [170000],
       ...,
       [148000],
       [124500],
       [137900]], dtype=int64)

## Decision Tree Regression

In [14]:
parameters = {
    "max_depth": np.linspace(1,11,11,endpoint=True), 
    "min_samples_split": [0.01,0.05, 0.1, 0.2]
}# TODO

dtr = DecisionTreeRegressor()# TODO
grid = GridSearchCV(dtr, parameters, cv=10) # TODO
grid.fit(train_x, train_y)


# summarize the results of the grid search
print("The best score is {}".format(grid.best_score_))
print("The best hyper parameter setting is {}".format(grid.best_params_))

The best score is 0.7239494832028728
The best hyper parameter setting is {'max_depth': 7.0, 'min_samples_split': 0.05}


C:\Users\deeks\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [15]:
#use the fine-tuned model to make predictions on the test data
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
test_z = grid.predict(test_x)

print("model Root Mean Squared Error: {}".format(np.sqrt(mean_squared_error(test_y, test_z))))
print("model Mean Absolute Error: {}".format(mean_absolute_error(test_y, test_z)))

model Root Mean Squared Error: 43259.74407782857
model Mean Absolute Error: 25969.475777280917


In [16]:
grid.score(test_x, test_y)

0.7453414348022471

## Random Forest Regression

In [17]:
parameters = {
    "n_estimators": [20, 200],
    "max_depth": [2, 4], 
    "min_samples_split": [0.05, 0.1, 0.2]
}

rfc_grid = GridSearchCV(RandomForestRegressor (n_jobs=-1, random_state=0), parameters, cv=3)
rfc_grid.fit(train_x, train)

# summarize the results of the grid search
print("The best score is {}".format(rfc_grid.best_score_))
print("The best hyper parameter setting is {}".format(rfc_grid.best_params_))

C:\Users\deeks\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\deeks\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\deeks\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\deeks\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y wa

C:\Users\deeks\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\deeks\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\deeks\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\deeks\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y wa

The best score is 0.7722869387226658
The best hyper parameter setting is {'max_depth': 4, 'min_samples_split': 0.05, 'n_estimators': 200}


In [18]:
# TODO: make prediction and evaluate the model performance on test data
#train_z = rfc_grid.predict(train_x)

test_z = rfc_grid.predict(test_x)

In [19]:
print("model Root Mean Squared Error: {}".format(np.sqrt(mean_squared_error(test_y, test_z))))
print("model Mean Absolute Error: {}".format(mean_absolute_error(test_y, test_z)))

model Root Mean Squared Error: 43270.03190966647
model Mean Absolute Error: 26118.04089727302


In [20]:
rfc_grid.score(test_x, test_y)

0.7452202969603534